# Feature Selection and LASSO

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [1]:
import graphlab as gl
from graphlab import SFrame

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
sales = SFrame('../data/kc_house_data.csv')

[INFO] GraphLab Create v1.8.3 started. Logging: /tmp/graphlab_server_1458562986.log


------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/tolo/Dropbox/Data/Studies/Informatics/ML Regression/assignments/data/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.15416 secs.

### Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

### Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = [
    'bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt',
    'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 
    'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = gl.linear_regression.create(
    sales, target='price', features=all_features, validation_set=None, 
    l2_penalty=0., l1_penalty=1e10, verbose=0)

Find what features had non-zero weight.

In [6]:
coeffs = model_all.coefficients
coeffs[coeffs['value'] != 0]

name,index,value,stderr
(intercept),None,274873.055652,None
bathrooms,None,8468.53105208,None
sqft_living,None,24.4207211037,None
sqft_living_sqrt,None,350.060551005,None
grade,None,842.068028819,None
sqft_above,None,20.0247226436,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

## Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(0.9, seed=1) 
# initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) 
# split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np
lams = np.logspace(1, 7, num=13)
lams

array([  1.00000000e+01,   3.16227766e+01,   1.00000000e+02,
         3.16227766e+02,   1.00000000e+03,   3.16227766e+03,
         1.00000000e+04,   3.16227766e+04,   1.00000000e+05,
         3.16227766e+05,   1.00000000e+06,   3.16227766e+06,
         1.00000000e+07])

In [9]:
def rss(model, data):
    return sum((model.predict(data) - data[model.target])**2)

def mape(model, data):
    return round(np.mean(list(abs(model.predict(data) - data[model.target])/data[model.target]))*100, 2)

In [70]:
models = []
for lam in lams:
    models += [gl.linear_regression.create(
        training, target='price', features=all_features, validation_set=None, 
        l2_penalty=0., l1_penalty=lam, verbose=0)]

In [71]:
def which_is_min(l):
    return l.index(min(l))

# [mape(m, validation) for m in models]
which_is_min([rss(m, validation) for m in models])

0

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [14]:
model = gl.linear_regression.create(
        training, target='price', features=all_features, validation_set=None, 
        l2_penalty=0, l1_penalty=lams[0], verbose=0)

In [15]:
rss(model, testing)

156983591307640.1

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [22]:
model.coefficients['value'].nnz()

18

## Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [19]:
max_nonzeros = 7

### Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [78]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [80]:
nonzeros = []
for l1p in l1_penalty_values:
    model = gl.linear_regression.create(
        training, target='price', features=all_features, validation_set=None, 
        l2_penalty=0, l1_penalty=l1p, verbose=0)
    nonzeros += [model.coefficients['value'].nnz()]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
which_l1_min = [i for i, n in enumerate(nonzeros) if n > max_nonzeros][-1]
which_l1_max = [i for i, n in enumerate(nonzeros) if n < max_nonzeros][0]
print which_l1_min, which_l1_max

In [82]:
l1_penalty_min = l1_penalty_values[which_l1_min]
l1_penalty_max = l1_penalty_values[which_l1_max]

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [83]:
print l1_penalty_min, l1_penalty_max

2976351441.63 3792690190.73


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [84]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [85]:
models = []
for l1p in l1_penalty_values:
    models += [gl.linear_regression.create(
        training, target='price', features=all_features, validation_set=None, 
        l2_penalty=0., l1_penalty=l1p, verbose=0)]
    

In [95]:
nonzeros = [m.coefficients['value'].nnz() for m in models]
rsss = [{i: rss(m, validation)} for i, m in enumerate(models) if nonzeros[i] == max_nonzeros]
min(rsss)

{11: 1046937489533986.2}

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [101]:
l1_penalty_values[11]

3448968612.1634369

In [98]:
coeffs = models[11].coefficients
coeffs[coeffs['value'] != 0]

name,index,value,stderr
(intercept),None,222253.191779,None
bedrooms,None,661.72256255,None
bathrooms,None,15873.9573905,None
sqft_living,None,32.4102220841,None
sqft_living_sqrt,None,690.114775638,None
grade,None,2899.42027112,None
sqft_above,None,30.011576021,None
